# Flash Evaluation on DARPA E3 Fivedirections Dataset: 

This notebook is specifically designed for the evaluation of Flash on the DARPA E3 Fivedirections dataset. Notably, the Fivedirections dataset is characterized as a node-level dataset. In our analysis, Flash is configured to operate in a node-level setting to aptly assess this dataset. A key aspect to note is that the Fivedirections dataset lacks certain essential node attributes for specific node types. This limitation means that Flash cannot be operated in a decoupled mode with offline GNN embeddings for this dataset. Consequently, we employ an online GNN coupled with word2vec semantic embeddings to achieve effective evaluation results for this dataset.

## Dataset Access: 
- Access the Fivedirections dataset via the following link: [Fivedirections Dataset](https://drive.google.com/drive/folders/1QlbUFWAGq3Hpl8wVdzOdIoZLFxkII4EK).
- The dataset files will be downloaded automatically by the script.

## Data Parsing and Execution:
- The script is designed to automatically parse the downloaded data files.
- Execute all cells within this notebook to obtain the evaluation results.

## Model Training and Execution Flexibility:
- The notebook is configured to use pre-trained model weights by default.
- It also provides the option to set parameters for independently training Graph Neural Networks (GNNs) and word2vec models.
- These newly trained models can then be utilized for a comprehensive evaluation of the dataset.

Adhere to these steps for a detailed and effective analysis of the Fivedirections dataset using Flash.


# Running Flash with pre-trained weights

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
import os
import torch.nn.functional as F
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
from torch_geometric.loader import NeighborLoader
import multiprocessing

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline

In [2]:
%%time

import gdown
urls = ["https://drive.google.com/file/d/1BeP80zUUmm4eZl0UuU43PsKNkl_xgskj/view?usp=drive_link"]
for url in urls:
    gdown.download(url, quiet=False, use_cookies=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1BeP80zUUmm4eZl0UuU43PsKNkl_xgskj
From (redirected): https://drive.google.com/uc?id=1BeP80zUUmm4eZl0UuU43PsKNkl_xgskj&confirm=t&uuid=b1ed96b9-7074-45b5-a667-f2c136e23599
To: /home/ubuntu/FlashExperiment/flash_ids/ta1-fivedirections-e3-official-2.json.tar.gz
100%|██████████| 12.4G/12.4G [13:52<00:00, 14.9MB/s]  

CPU times: user 12.7 s, sys: 13.5 s, total: 26.2 s
Wall time: 13min 57s


In [3]:
Train = False

In [4]:
from pprint import pprint
import gzip
from sklearn.manifold import TSNE
import json
import copy
import os

In [5]:
import re

def extract_uuid(line):
    pattern_uuid = re.compile(r'uuid\":\"(.*?)\"')
    return pattern_uuid.findall(line)

def extract_subject_type(line):
    pattern_type = re.compile(r'type\":\"(.*?)\"')
    return pattern_type.findall(line)

def show(file_path):
    print(f"Processing {file_path}")

def extract_edge_info(line):
    pattern_src = re.compile(r'subject\":{\"com.bbn.tc.schema.avro.cdm18.UUID\":\"(.*?)\"}')
    pattern_dst1 = re.compile(r'predicateObject\":{\"com.bbn.tc.schema.avro.cdm18.UUID\":\"(.*?)\"}')
    pattern_dst2 = re.compile(r'predicateObject2\":{\"com.bbn.tc.schema.avro.cdm18.UUID\":\"(.*?)\"}')
    pattern_type = re.compile(r'type\":\"(.*?)\"')
    pattern_time = re.compile(r'timestampNanos\":(.*?),')

    edge_type = extract_subject_type(line)[0]
    timestamp = pattern_time.findall(line)[0]
    src_id = pattern_src.findall(line)

    if len(src_id) == 0:
        return None, None, None, None, None

    src_id = src_id[0]
    dst_id1 = pattern_dst1.findall(line)
    dst_id2 = pattern_dst2.findall(line)

    if len(dst_id1) > 0 and dst_id1[0] != 'null':
        dst_id1 = dst_id1[0]
    else:
        dst_id1 = None

    if len(dst_id2) > 0 and dst_id2[0] != 'null':
        dst_id2 = dst_id2[0]
    else:
        dst_id2 = None

    return src_id, edge_type, timestamp, dst_id1, dst_id2

def process_data(file_path):
    id_nodetype_map = {}
    notice_num = 1000000
    for i in range(100):
        now_path = file_path + '.' + str(i)
        if i == 0:
            now_path = file_path
        if not os.path.exists(now_path):
            break

        with open(now_path, 'r') as f:
            show(now_path)
            cnt = 0
            for line in f:
                cnt += 1
                if cnt % notice_num == 0:
                    print(cnt)

                if 'com.bbn.tc.schema.avro.cdm18.Event' in line or 'com.bbn.tc.schema.avro.cdm18.Host' in line:
                    continue

                if 'com.bbn.tc.schema.avro.cdm18.TimeMarker' in line or 'com.bbn.tc.schema.avro.cdm18.StartMarker' in line:
                    continue

                if 'com.bbn.tc.schema.avro.cdm18.UnitDependency' in line or 'com.bbn.tc.schema.avro.cdm18.EndMarker' in line:
                    continue

                uuid = extract_uuid(line)[0]
                subject_type = extract_subject_type(line)

                if len(subject_type) < 1:
                    if 'com.bbn.tc.schema.avro.cdm18.MemoryObject' in line:
                        id_nodetype_map[uuid] = 'MemoryObject'
                        continue
                    if 'com.bbn.tc.schema.avro.cdm18.NetFlowObject' in line:
                        id_nodetype_map[uuid] = 'NetFlowObject'
                        continue
                    if 'com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject' in line:
                        id_nodetype_map[uuid] = 'UnnamedPipeObject'
                        continue

                id_nodetype_map[uuid] = subject_type[0]

    return id_nodetype_map

def process_edges(file_path, id_nodetype_map):
    notice_num = 1000000
    not_in_cnt = 0

    for i in range(100):
        now_path = file_path + '.' + str(i)
        if i == 0:
            now_path = file_path
        if not os.path.exists(now_path):
            break

        with open(now_path, 'r') as f, open(now_path+'.txt', 'w') as fw:
            cnt = 0
            for line in f:
                cnt += 1
                if cnt % notice_num == 0:
                    print(cnt)

                if 'com.bbn.tc.schema.avro.cdm18.Event' in line:
                    src_id, edge_type, timestamp, dst_id1, dst_id2 = extract_edge_info(line)

                    if src_id is None or src_id not in id_nodetype_map:
                        not_in_cnt += 1
                        continue

                    src_type = id_nodetype_map[src_id]

                    if dst_id1 is not None and dst_id1 in id_nodetype_map:
                        dst_type1 = id_nodetype_map[dst_id1]
                        this_edge1 = f"{src_id}\t{src_type}\t{dst_id1}\t{dst_type1}\t{edge_type}\t{timestamp}\n"
                        fw.write(this_edge1)

                    if dst_id2 is not None and dst_id2 in id_nodetype_map:
                        dst_type2 = id_nodetype_map[dst_id2]
                        this_edge2 = f"{src_id}\t{src_type}\t{dst_id2}\t{dst_type2}\t{edge_type}\t{timestamp}\n"
                        fw.write(this_edge2)

def run_data_processing():
    os.system('tar -zxvf ta1-fivedirections-e3-official-2.json.tar.gz')

    path_list = ['ta1-fivedirections-e3-official-2.json']

    for path in path_list:
        id_nodetype_map = process_data(path)
        process_edges(path, id_nodetype_map)

    os.system('cp ta1-fivedirections-e3-official-2.json.txt fivedirections_train.txt')
    os.system('cp ta1-fivedirections-e3-official-2.json.23.txt fivedirections_test.txt')

In [6]:
%%time

run_data_processing()

ta1-fivedirections-e3-official-2.json
ta1-fivedirections-e3-official-2.json.1
ta1-fivedirections-e3-official-2.json.10
ta1-fivedirections-e3-official-2.json.11
ta1-fivedirections-e3-official-2.json.12
ta1-fivedirections-e3-official-2.json.13
ta1-fivedirections-e3-official-2.json.14
ta1-fivedirections-e3-official-2.json.15
ta1-fivedirections-e3-official-2.json.16
ta1-fivedirections-e3-official-2.json.17
ta1-fivedirections-e3-official-2.json.18
ta1-fivedirections-e3-official-2.json.19
ta1-fivedirections-e3-official-2.json.2
ta1-fivedirections-e3-official-2.json.20
ta1-fivedirections-e3-official-2.json.21
ta1-fivedirections-e3-official-2.json.22
ta1-fivedirections-e3-official-2.json.23
ta1-fivedirections-e3-official-2.json.24
ta1-fivedirections-e3-official-2.json.25
ta1-fivedirections-e3-official-2.json.26
ta1-fivedirections-e3-official-2.json.27
ta1-fivedirections-e3-official-2.json.28
ta1-fivedirections-e3-official-2.json.29
ta1-fivedirections-e3-official-2.json.3
ta1-fivedirections-e3-

In [7]:
def prepare_graph(df):
    nodes = {}
    labels = {}
    edges = []
    
    dummies = {'SUBJECT_PROCESS': 0, 'FILE_OBJECT_CHAR': 1, 'VALUE_TYPE_SRC': 2, 'SRCSINK_DATABASE': 3,
               'FILE_OBJECT_UNIX_SOCKET': 4,'FILE_OBJECT_BLOCK': 5, 'NetFlowObject': 6, 
               'SRCSINK_PROCESS_MANAGEMENT': 7, 'SUBJECT_THREAD': 8}

    for i in range(len(df)):
        x = df.iloc[i]
        if x['object'] in dummies:
            action = x["action"]

            actorid = x["actorID"]
            if not (actorid in nodes):
                nodes[actorid] =  []
            if x['exec'] != '':
                nodes[actorid].append(x['exec'])
            nodes[actorid].append(action)
            if x['path'] != '':
                nodes[actorid].append(x['path'])
            labels[actorid] = dummies[x['actor_type']]

            objectid = x["objectID"]
            if not (objectid in nodes):
                nodes[objectid] =  []
            if x['exec'] != '':
                nodes[objectid].append(x['exec'])
            nodes[objectid].append(action)
            if x['path'] != '':
                 nodes[objectid].append(x['path'])
            labels[objectid] = dummies[x['object']]

            edges.append(( actorid, objectid, action ))

    features = []
    feat_labels = []
    edge_index = [[],[]]
    index  = {}
    mapp = []

    for k,v in nodes.items():
      features.append(v)
      feat_labels.append(labels[k])
      index[k] = len(features) - 1
      mapp.append(k)

    for x in edges:
        src = index[x[0]]
        dst = index[x[1]]
        
        edge_index[0].append(src)
        edge_index[1].append(dst)
    
    print("At the end of function prepare_graph, |nodes|:", len(nodes), ", |labels|:", len(labels), ", |edges|:", len(edges))

    return features,feat_labels,edge_index,mapp

In [8]:
import random 

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [9]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv, GATConv
import torch.nn as nn

class GCN(torch.nn.Module):
    def __init__(self,in_channel,out_channel):
        super().__init__()
        self.conv1 = SAGEConv(in_channel, 32, normalize=True)
        self.conv2 = SAGEConv(32, out_channel, normalize=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        return x

In [10]:
from gensim.models.callbacks import CallbackAny2Vec
import gensim
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm
import time

class EpochSaver(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        model.save('trained_weights/fivedirections/word2vec_five_E3.model')
        self.epoch += 1

In [11]:
class EpochLogger(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [12]:
logger = EpochLogger()
saver = EpochSaver()

In [13]:
def add_attributes(d,p):
    
    f = open(p)
    data = [json.loads(x) for x in f if "EVENT" in x]

    info = []
    for x in data:
        try:
            action = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['type']
        except:
            action = ''
        try:
            actor = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['subject']['com.bbn.tc.schema.avro.cdm18.UUID']
        except:
            actor = ''
        try:
            obj = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObject']['com.bbn.tc.schema.avro.cdm18.UUID']
        except:
            obj = ''
        try:
            timestamp = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['timestampNanos']
        except:
            timestamp = ''
        try:
            cmd = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['properties']['map']['exec']
        except:
            cmd = ''
        try:
            path = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObjectPath']['string']
        except:
            path = ''
        try:
            path2 = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObject2Path']['string']
        except:
            path2 = ''
        try:
            obj2 = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObject2']['com.bbn.tc.schema.avro.cdm18.UUID']
            info.append({'actorID':actor,'objectID':obj2,'action':action,'timestamp':timestamp,'exec':cmd, 'path':path2})
        except:
            pass

        info.append({'actorID':actor,'objectID':obj,'action':action,'timestamp':timestamp,'exec':cmd, 'path':path})

    rdf = pd.DataFrame.from_records(info).astype(str)
    d = d.astype(str)

    return d.merge(rdf,how='inner',on=['actorID','objectID','action','timestamp']).drop_duplicates()

In [14]:
%%time

if Train:
    f = open("fivedirections_train.txt")
    data = f.read().split('\n')
    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df = df.dropna()
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = add_attributes(df,"ta1-fivedirections-e3-official-2.json")
    phrases,labels,edges,mapp = prepare_graph(df)

CPU times: user 4 μs, sys: 0 ns, total: 4 μs
Wall time: 7.15 μs


In [15]:
from sklearn.utils import class_weight
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

model = GCN(30,9).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [16]:
%%time

if Train:
    word2vec = Word2Vec(sentences=phrases, vector_size=30, window=5, min_count=1, workers=8,epochs=300,callbacks=[saver,logger])

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 7.39 μs


In [17]:
import math
import torch
import numpy as np
from gensim.models import Word2Vec

class PositionalEncoder:

    def __init__(self, d_model, max_len=100000):
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        self.pe = torch.zeros(max_len, d_model)
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)

    def embed(self, x):
        return x + self.pe[:x.size(0)]


def infer(document):
    word_embeddings = [w2vmodel.wv[word] for word in document if word in  w2vmodel.wv]
    
    if not word_embeddings:
        return np.zeros(20)

    output_embedding = torch.tensor(word_embeddings, dtype=torch.float)
    if len(document) < 100000:
        output_embedding = encoder.embed(output_embedding)

    output_embedding = output_embedding.detach().cpu().numpy()
    return np.mean(output_embedding, axis=0)

encoder = PositionalEncoder(30)
w2vmodel = Word2Vec.load("trained_weights/fivedirections_original/word2vec_five_E3.model")

In [18]:
w2vmodel

In [19]:
%%time

from torch_geometric import utils

# Fixed RuntimeError: indices should be either on cpu or on the same device as the indexed tensor (cpu)

if Train:
    l = np.array(labels)
    class_weights = class_weight.compute_class_weight(class_weight = "balanced",classes = np.unique(l),y = l)
    class_weights = torch.tensor(class_weights,dtype=torch.float).to(device)
    criterion = CrossEntropyLoss(weight=class_weights,reduction='mean')

    nodes = [infer(x) for x in phrases]
    nodes = np.array(nodes)  

    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes).to(device)
    mask = torch.tensor([True]*graph.num_nodes, dtype=torch.bool, device=device)

    for m_n in range(13):

        loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
        total_loss = 0
        for subg in loader:
            subg=subg.to(device)
            model.train()
            optimizer.zero_grad() 
            out = model(subg.x, subg.edge_index) 
            loss = criterion(out, subg.y) 
            loss.backward() 
            optimizer.step()      
            total_loss += loss.item() * subg.batch_size
        print(total_loss / mask.sum().item())

        loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
        for subg in loader:
            subg=subg.to(device)
            model.eval()
            out = model(subg.x, subg.edge_index)

            sorted, indices = out.sort(dim=1,descending=True)
            conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
            conf = (conf - conf.min()) / conf.max()

            pred = indices[:,0]
            cond = (pred == subg.y) | (conf >= 0.9)
            subg_n_id_cond = subg.n_id[cond].to(device)
            mask[subg_n_id_cond] = False

        torch.save(model.state_dict(), f'trained_weights/fivedirections/lword2vec_gnn_five{m_n}_E3.pth')
        print(f'Model# {m_n}. {mask.sum().item()} nodes still misclassified \n')

CPU times: user 10 μs, sys: 1e+03 ns, total: 11 μs
Wall time: 15 μs


In [20]:
from itertools import compress
from torch_geometric import utils

def Get_Adjacent(ids, mapp, edges, hops):
    if hops == 0:
        return set()
    
    neighbors = set()
    for edge in zip(edges[0], edges[1]):
        if any(mapp[node] in ids for node in edge):
            neighbors.update(mapp[node] for node in edge)

    if hops > 1:
        neighbors = neighbors.union(Get_Adjacent(neighbors, mapp, edges, hops - 1))
    
    return neighbors

def calculate_metrics(TP, FP, FN, TN):
    FPR = FP / (FP + TN) if FP + TN > 0 else 0
    TPR = TP / (TP + FN) if TP + FN > 0 else 0

    prec = TP / (TP + FP) if TP + FP > 0 else 0
    rec = TP / (TP + FN) if TP + FN > 0 else 0
    fscore = (2 * prec * rec) / (prec + rec) if prec + rec > 0 else 0
    acc = (TN + TP) / (TN + TP + FN + FP) if TN + TP + FN + FP > 0 else 0

    return prec, rec, fscore, FPR, TPR, acc

def helper(MP, all_pids, GP, edges, mapp):
    TP = MP.intersection(GP)
    FP = MP - GP
    FN = GP - MP
    TN = all_pids - (GP | MP)

    two_hop_gp = Get_Adjacent(GP, mapp, edges, 2)
    two_hop_tp = Get_Adjacent(TP, mapp, edges, 2)
    FPL = FP - two_hop_gp
    TPL = TP.union(FN.intersection(two_hop_tp))
    FN = FN - two_hop_tp

    TP, FP, FN, TN = len(TPL), len(FPL), len(FN), len(TN)

    prec, rec, fscore, FPR, TPR, acc = calculate_metrics(TP, FP, FN, TN)
    # print(f"True Positives: {TP}, False Positives: {FP}, False Negatives: {FN}")
    print(f"True Positives/False Positives/False Negatives/True Negatives: {TP}/{FP}/{FN}/{TN}")
    print(f"Accuracy: {round(acc, 2)}, Precision: {round(prec, 2)}, Recall: {round(rec, 2)}, Fscore: {round(fscore, 2)}")
    
    return TPL, FPL

In [21]:
%%time

f = open("fivedirections_test.txt")
data = f.read().split('\n')
data = [line.split('\t') for line in data]
df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
df = df.dropna()
df.sort_values(by='timestamp', ascending=True,inplace=True)

df = add_attributes(df,"ta1-fivedirections-e3-official-2.json.23")

CPU times: user 2min 57s, sys: 26.3 s, total: 3min 24s
Wall time: 3min 23s


In [22]:
%%time

data = df
phrases,labels,edges,mapp = prepare_graph(data)
nodes = [infer(x) for x in phrases]
nodes = np.array(nodes)

with open("data_files/fivedirections.json", "r") as json_file:
    GT_mal = json.load(json_file)
    
GT_mal = set([x for x in GT_mal if x in mapp])

all_ids = list(data['actorID']) + list(data['objectID'])
all_ids = set(all_ids)

print(len(all_ids))

At the end of function prepare_graph, |nodes|: 21676 , |labels|: 21676 , |edges|: 880826
374484
CPU times: user 2min 9s, sys: 0 ns, total: 2min 9s
Wall time: 2min 5s


In [23]:
%%time

# Fixed RuntimeError: indices should be either on cpu or on the same device as the indexed tensor (cpu)

graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
graph.n_id = torch.arange(graph.num_nodes).to(device)
flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool, device=device)

for m_n in range(13):
  model.load_state_dict(torch.load(f'trained_weights/fivedirections_original/lword2vec_gnn_five{m_n}_E3.pth',map_location=torch.device('cpu')))
  loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000)    
  for subg in loader:
      subg=subg.to(device)
      model.eval()
      out = model(subg.x, subg.edge_index)

      sorted, indices = out.sort(dim=1,descending=True)
      conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
      conf = (conf - conf.min()) / conf.max()
    
      pred = indices[:,0]
      cond = (pred == subg.y) & (conf > 0.94)
      subg_n_id_cond = subg.n_id[cond].to(device)
      flag[subg_n_id_cond] = torch.logical_and(flag[subg_n_id_cond], torch.tensor([False]*len(flag[subg_n_id_cond]), dtype=torch.bool, device=device))

index = utils.mask_to_index(flag).tolist()
ids = set([mapp[x] for x in index])
alerts = helper(set(ids),set(all_ids),GT_mal,edges,mapp) 

True Positives/False Positives/False Negatives/True Negatives: 395/150/30/359093
Accuracy: 1.0, Precision: 0.72, Recall: 0.93, Fscore: 0.81
CPU times: user 8.32 s, sys: 0 ns, total: 8.32 s
Wall time: 4.03 s


In [24]:
!nvidia-smi

Tue Dec 24 09:18:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:1E.0 Off |                    0 |
| N/A   33C    P0              26W /  70W |    401MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--